# 分布式Redis集群

[Redis集群](https://github.com/kubernetes/kubernetes/tree/master/examples/redis) 是官方提供的案例，用来展示Kubernetes的特色。

一个Redis集群当中包含一个Master节点，多个Slave节点，以及数个Sentinel节点。通过Sentinel节点实现了health checking以及failover，规避了单点故障。让我们试着在集群当中把它跑起来。

## 部署Redis Master

我们先来看看`redis-master`的内容

In [ ]:
!cat redis-master.yaml

可以看到，在`redis-master`这个Pod里面，部署了两个容器，分别是一个Master，一个Sentinel。因为他们共享同一个网络环境，所以Sentinel可以用`$(hostname -i):6379`来找到Master，将它加入到监控列表当中，完成初始化。

使用`kubectl`完成创建

In [ ]:
!kubectl create -f redis-master.yaml

## 创建Sentinel服务

Sentinel在Redis集群当中非常重要，当用户需要访问Redis集群时，先访问Sentinel，获取当前的Master和Slave地址，再进一步完成访问。

我们将Sentinel暴露到集群当中去

In [ ]:
!kubectl create -f redis-sentinel-service.yaml

## 创建Replication Controller

上面创建的`redis-master`仅有一个Master，一个Sentinel，还远远称不上集群。让我们通过Replication Controller来增加更多的节点。

In [ ]:
!kubectl create -f redis-controller.yaml

In [ ]:
!kubectl create -f redis-sentinel-controller.yaml

需要指出的是，`redis-master`已经满足上述Replication Controller中Selector的条件，而他们的Replica数均为1，所以它们并不会创建新的Pod，让我们来检查一下

In [ ]:
!kubectl get pods

这里体现了Replication Controller的工作原理，它仅仅是检查Selector得到的Pod数和目标数是否一致，如果不一致才会采取相应动作。

让我们来创建更多的Replica

In [ ]:
!kubectl scale rc redis --replicas=3

In [ ]:
!kubectl scale rc redis-sentinel --replicas=3

这些新创建的Redis服务器和Sentinel服务器，将通过刚才暴露给集群的`redis-sentinel`服务，找到现有的Sentinel服务器，并加入集群。

## 删除手动创建的Pod

最开始的`redis-master`是我们手动创建的，现在我们将它移除，而让集群自动管理。

In [ ]:
!kubectl delete pods redis-master

这时，上面的两个Replication Controller会检查到Redis服务器和Sentinel服务器均只剩下两台，然后开始自动创建。而集群内部由于Master节点被移除，开始了选举流程，会有一个新的Redis服务器被选举为Master节点。

让我们来验证一下

In [ ]:
!kubectl get pods

In [ ]:
!kubectl logs $SENTINEL_POD redis-sentinel

## 总结

通过上面简单几步，我们就在Kubernetes集群上搭建起了分布式的Redis集群。当有服务器故障时，Redis集群会利用Kubernetes的自动管理功能而恢复，服务并不会中断。当需要扩展Redis性能时，可以简单的下达scale指令增加Redis服务器数量，来承载更多的读操作。

需要指出的是Redis集群并不提供Sharding的功能，所有的写操作都会发送到单一的Master节点上，所以它的主要功能是解决高可用的问题，虽然可以通过读写分离，Slave数量动态增加解决部分规模扩展的问题，但是并不全面。

如果对具体的细节感兴趣，可以查看启动脚本文件

In [ ]:
!cat image/run.sh

## 资源释放

释放案例当中创建的资源

In [ ]:
!kubectl delete rc redis

In [ ]:
!kubectl delete rc redis-sentinel

In [ ]:
!kubectl delete svc redis-sentinel